In [23]:
import pandas as pd

In [24]:
room_df = pd.read_csv('building_area/APMRooms.csv')

In [33]:
#Returns 
def extract_buildingData(bd_filepath, building_name):
    room_df = pd.read_csv(bd_filepath)
    room_df = room_df[room_df['Category'].str.match('A - Assignable Area')][['Building', 
                                                                             'Room', 
                                                                             'Area', 
                                                                             'Room Use (S)',
                                                                             'Stations']]

    if building_name != room_df[['Building']]['Building'].unique()[0]:
        print('Error: Building Name of input file does not match desired Building Name')
        print('Building Name from file is {}'.format(room_df[['Building']]))
    room_df = room_df[room_df['Room Use (S)'].str.contains('Storage', regex=True) == False]
    return room_df

![](images/ashrae2.png)

![](images/ashrae3.png)

![](images/ashrae.png)

In [34]:
room_df = extract_buildingData('building_area/APMRooms.csv', 'Applied Physics and Mathematics')

In [36]:
room_df.head()

,Building,Room,Area,Room Use (S),Stations
38,Applied Physics and Mathematics,B301,172,560 - Media Production (S),1.0
39,Applied Physics and Mathematics,B301A,335,560 - Media Production (S),0.0
40,Applied Physics and Mathematics,B301B,194,560 - Media Production (S),0.0
41,Applied Physics and Mathematics,B302,335,560 - Media Production (S),1.0
42,Applied Physics and Mathematics,B302A,44,560 - Media Production (S),0.0


In [40]:
def impute_buildingData(room_df):
    #Based on the area we can imput airflow given ANSI/ASHRAE Standards
    #url: https://www.ashrae.org/File%20Library/Technical%20Resources/Standards%20and%20Guidelines/Standards%20Addenda/62.1-2016/62_1_2016_s_20190726.pdf
    #Formula: Ventilation = Rp * Pz + Ra * Az
    #Az = Floor area in sq ft
    #Pz = Zone occupancy or zone population 
    #Rp = Outdoor Airflow rate in cfm / person (See ASHRAE table)
    #Ra = Outdoor Airflow rate in cfm /sq ft (See ASHRAE table) 
    #Od = Occupant Density. #occ/1000 sq ft(See ASHRAE Table)
    ASHRAE_table = {
        'lectureClassroom': {'Rp': 7.5, 'Ra': .06, 'Od': 65},
        'lectureHall': {'Rp': 7.5, 'Ra': .06, 'Od': 150},
        'collegeLaboratories': {'Rp': 10, 'Ra': .18, 'Od': 25},
        'officeSpace': {'Rp': 5, 'Ra': .06, 'Od': 5},
        'libraries' : {'Rp': 5, 'Ra': .12, 'Od': 10}
    }
    CFMs = []
    for index, row in room_df.iterrows():
        #If specified room is a designated as a classroom
        #Then impute classroom CFM given area
        r_use = row['Room Use (S)']
        Az = row['Area']
        Pz = row['Stations']
        if r_use in ['110 - Classroom (S)']:
            if Pz == 0:
                Pz = row['Area'] * ASHRAE_table['lectureHall']['Od']
            Rp = ASHRAE_table['lectureHall']['Rp']
            Ra = ASHRAE_table['lectureHall']['Ra']
            room_CFM = Rp * Pz + Ra * Az
            CFMs.append(room_CFM)    
        #Office space
        if r_use in ['211 - Research Office - Enclosed (S)',
                     '335 - Office Service (S)',
                     '320 - Administrative Office - Enclosed (S)', 
                     '211 - Research Office - Open (S)',
                     '310 - Academic Office - Enclosed (S)',
                     '226 - Research Office Service (S)',
                     '320 - Administrative Office - Open (S)']:
            if Pz == 0:
                Pz = row['Area'] * ASHRAE_table['officeSpace']['Od']
            Rp = ASHRAE_table['officeSpace']['Rp']
            Ra = ASHRAE_table['officeSpace']['Ra']
            room_CFM = Rp * Pz + Ra * Az
            CFMs.append(room_CFM)            
        else:
            print('New Classification Required:')
            print(str(r_use))
            
    return room_df.assign(CFM = CFMs)

In [41]:
impute_buildingData(room_df)

New Classification Required:
560 - Media Production (S)
New Classification Required:
560 - Media Production (S)
New Classification Required:
560 - Media Production (S)
New Classification Required:
560 - Media Production (S)
New Classification Required:
560 - Media Production (S)
New Classification Required:
270 - Open Laboratory (N)
New Classification Required:
270 - Open Lab - Restricted (N)
New Classification Required:
340 - Conference Room (S)
New Classification Required:
340 - Conference Room (S)
New Classification Required:
270 - Open Laboratory (N)
New Classification Required:
710 - Shop - General and Research (S)
New Classification Required:
715 - Shop Service- General & Research (S)
New Classification Required:
345 - Conference Room Service (S)
New Classification Required:
340 - Conference Room (S)
New Classification Required:
710 - Shop - General and Research (S)
New Classification Required:
710 - Shop - General and Research (S)
New Classification Required:
710 - Shop - Genera

ValueError: Length of values does not match length of index

In [ ]:
def append_roomData(room_df, Master_room_df):
    